## Generic setup for the CW Nano

In [9]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'
CRYPTO_TARGET = 'NONE'
%run "../../../Helper_Scripts/Setup_Generic.ipynb"
scope.adc.samples = 5000
scope._cwusb.usbtx.rep = 0x81
scope._cwusb.usbtx.wep = 0x02

INFO: Found ChipWhisperer😍


## Calling The path for the hex file and programming the board

In [10]:
fw_path = "../calc-CWNANO.hex"
cw.program_target(scope, cw.programmers.STM32FProgrammer, fw_path)

Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 13531 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 13531 bytes


Reboot function is used to manipulate the array bytes.

In [11]:
def reboot_flush():
    scope.io.nrst = False
    time.sleep(0.05)
    scope.io.nrst = "high_z"
    time.sleep(0.05)
    #Flush garbage too
    target.flush()

#############################################
There are two possible attack can be applied in this operation. 

1# CPA attack. However, The variation is very less. Even we found some bytes but the attack is not strong enough. Also, CPA has many calculation and comperatively more execution time.

2# Buffer overflow with addition and substraction. As soon as the buffer overflow happens there is a power drop. We are focusing ion this one.

#############################################

The main assumtion of this attack is that buffer overflow. That will be used as the vulnerability.

For example: A byte has the value 255 as 1111 1111. As soon as we add 1 it will be 1 0000 000. Here, The hamming weight will drop from 8 to 1. This will have a visible impact in the power. Similarly, if we substract 1 from 0000 0000 it will generate 1111 1111. There wil be similar impact.

Now, as the calculator hold the memory, and we have the option to manipulate the memory by performing calculation. We can use this function to manipulate the data inside.

The C code has a array as a int. Which is 32 bit (4 byte) and array has 3 elements. Difficulty to attack this program with brut force is 2^(32*32*32). Which is very hard or even not possible considering time and resources.

To simplify the attack, we only want to attack a byte of the array elemants. Therefor, we have only need to attck 4 byte in 4 loop.

Steps:
    
    1# We need to get rid of other bytes except the one we are attacking. 
    
    For example: if are attcking byte 0. Then we can remove the byte 3-1 as shifting the byte 0, 3*8 times to left. This will 
    keep the byte 0 to byte3. Then as we are using lsb byte to perform the operation, we need to shift the byte0 from msb 
    byte3. We can shift it 3*8 times to right. This will only keep the byte0 in the array. However baserd on the msb bit of the 
    leftover byte it will padd 1s or 0s in other3 msb bytes. Which is a advantage for the attcker to perform buffer overflow
    with sudden power drop/rise.        
    Main array-->  |<byte3>|<byte2>|<byte1>|<byte0>| 
    Shift left-->  |<byte0>|<*****>|<*****>|<*****>| (Shift left can be done by multiply by 2 for one shift right)
    Shift right->  |<*1/0*>|<*1/0*>|<*1/0*>|<byte0>| (Shift right can be done by dividing by 2 for one shift left)
    
    
    
    2# To get a higher power drop, we need a transaction of 0000 0000 to 1111 1111 or vice versa.
    therefor we will collect the power traces for addition of 0 to 255 as well as substraction 0 to 255.
    
    3. Repeat step 1-2 for each byte and collect the power traces for each byte.
    ##Note that, The power traces should be less noisy. Therefore we can collect the traces multiple times for same operation 
    and get the mean of thiose to decrerase the effect of noise. For our case we used 20 repeted trace collection. (Less is 
    also ok.)
        
    4. Perform the attack on the power traces and find the possible keys.
    
    
    



In [1]:
#####   Attention #####
#Run this code if you only need to take the power traces.
#Otherwise, Power traces are already pre-recorded and saved in the numpy files in this directory for the given hex file
#Recording power traces need little time. As we record multiple times for each byte as well as for reducing the noise.

import time
from tqdm.notebook import tqdm, trange
import numpy as np
import matplotlib.pyplot as plt

for byte in range (1,5):
    print("Byte:",byte)
    reboot_flush()

    #change the value 1 to 4  at a time and save the power traces


    for i in range ((4-byte)*4):
        msg=[2**2]     
        scope.arm()
        target.simpleserial_write('m',bytearray(msg))  
        scope.capture()

    for i in range (3*4):
        msg=[2**2]     
        scope.arm()
        target.simpleserial_write('d',bytearray(msg))  
        scope.capture()


    T=40
    D= 256

    repeat=20

    print("############## Substraction #####################")    

    base_power_substract=np.zeros([D,T],dtype='float')

    for i in trange (D):
        msg=[i % 256] 

        repeated_trace=np.zeros([repeat,T],dtype='float')
        for j in range (repeat): 

            scope.arm()
            target.simpleserial_write('s',bytearray(msg))    
            scope.capture()
            repeated_trace[j,:]=scope.get_last_trace()[0:T] 


            scope.arm()
            target.simpleserial_write('p',bytearray(msg))
            scope.capture()

        base_power_substract[i,:]=np.mean(repeated_trace[:,:], axis=0)


    np.save("calc_new_pow_byte%d_sub.npy" %byte,base_power_substract)   
    print("############## Addition #####################")    
    ###
    base_power_plus=np.zeros([D,T],dtype='float')

    for i in trange (D):
        msg=[i % 256] 

        repeated_trace=np.zeros([repeat,T],dtype='float')
        for j in range (repeat): 

            scope.arm()
            target.simpleserial_write('p',bytearray(msg))             
            scope.capture()
            repeated_trace[j,:]=scope.get_last_trace()[0:T]         

            scope.arm()
            target.simpleserial_write('s',bytearray(msg))
            scope.capture()

        base_power_plus[i,:]=np.mean(repeated_trace[:,:], axis=0)
    np.save("calc_new_pow_byte%d_plus.npy" %byte,base_power_plus)   



Byte: 1


NameError: name 'reboot_flush' is not defined

#################################################

Now, we will attack on the pre-recorded power traces.

We found that, if the byte value is positive, I mean if 0-127 then we get 0 padded in msb bytes after getting rid of other bytes. However if the byte value is negative or 128-255 then we get 1 padded in msb bits. 

As we dont know the byte value is positive or negative or MSB is 1 or 0. We need to consider both cases to find the byte value. We will use power for substraction for find the values related to 0-127 and addition or plus for finding the value between 128-255.

During attack for each byte for power between 0-255 we will perform correlation with previous power. If, we find that the correlation is high then it means that no power drop/rise for buffer overflow. However, as soon as we see that there is a high drop of correlation comparing other index, its the memory value.

Then, for ploting purposes we can make it reverse by substrcating from 1. This will show that there will be a high pick ,in one index, that will be the memory byte.

By repeating this for every byte we will get an 12 bytes array for the memory element. However, as we dont know the value is positive or negative, we will have 2 12 byte arrays. Based on the high corelation, between two arrays we will chose the one element for each indexes and find the final memory. 

After finding the guessed elements its time to verify..


# Attention: please change "base_power_substract" as needed


In [12]:
from tqdm.notebook import tqdm, trange
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import numpy as np
import time

T=40
D= 256
k_possible=255
t_init_list=[23,27,32]
t_range=5

memory_positive=np.zeros([12], dtype='int')
memory_negative=np.zeros([12], dtype='int')

memory_pos_corr=np.zeros([12], dtype='float')
memory_neg_corr=np.zeros([12], dtype='float')

picks=np.zeros([2,12], dtype='int')
msg=np.zeros([12],dtype='int')



for byte in range (1,5):
    #Loading the Power
    
    ### For new power traces###
    #base_power_substract=np.load("calc_new_pow_byte%d_sub.npy" %byte)
    #base_power_plus=np.load("calc_new_pow_byte%d_plus.npy" %byte)
    
    ### For prerecorded power traces ###
    base_power_substract=np.load("calc_pow_byte%d_sub.npy" %byte)
    base_power_plus=np.load("calc_pow_byte%d_plus.npy" %byte)


    r=np.zeros([k_possible, 1], dtype='float')
    base_power=base_power_substract

    for arr in range (3):
        t_init=t_init_list[arr]


        for i in range (D-1):
            r[i,0], _ = pearsonr(base_power[i,t_init:t_init+t_range], base_power[i+1,t_init:t_init+t_range])

        corr=1-(np.abs(r));
        guessed_key_array=np.argsort(np.abs(corr),axis=0)
        found_key=guessed_key_array[-1,0]
        max_corr=max(corr)
        
        memory_pos_corr[(byte-1)+arr*4]=max_corr
        memory_positive[(byte-1)+arr*4]=found_key
        picks[0,(byte-1)+arr*4] = (corr > max(corr)*0.20).sum()

        #print('Byte:'+str(byte)+' Gussed key:' +str(found_key)+" Corr: %f " %(max_corr))

        
        ##### Uncomment the plotting if you want to see the plots 
        '''plt.figure(figsize=(18.5, 10.5), dpi=150)
        axes = plt.gca()
        plt.plot(corr)
        plt.title('Byte:'+str(byte)+' Arr:'+str(arr)+' Positive Gussed key:' +str(found_key)+" range: %d to %d" %(t_init,t_init+t_range))
        plt.xticks(np.arange(0, len(corr)+1, 10))'''
        #######################################################


    r=np.zeros([k_possible, 1], dtype='float')
    base_power=base_power_plus   
        
    for arr in range (3):   
        base_power=base_power_plus
        t_init=t_init_list[arr]

        for i in range (D-1):
            r[i,0], _ = pearsonr(base_power[i,t_init:t_init+t_range], base_power[i+1,t_init:t_init+t_range])

        corr=1-(np.abs(r));
        guessed_key_array=np.argsort(np.abs(corr),axis=0)
        found_key=255-guessed_key_array[-1,0]-1
        max_corr=max(corr)
        #print('Byte:'+str(byte)+' Gussed key:' +str(found_key)+" Corr: %f " %(max_corr))
        memory_neg_corr[(byte-1)+arr*4]=max_corr
        memory_negative[(byte-1)+arr*4]=found_key
        picks[1,(byte-1)+arr*4] = (corr > max(corr)*0.20).sum()

        ##### Uncomment the plotting if you want to see the plots 
        '''plt.figure(figsize=(18.5, 10.5), dpi=150)
        axes = plt.gca()
        plt.plot(corr)
        plt.title('Byte:'+str(byte)+' Arr:'+str(arr)+' Negative Gussed key:' +str(found_key)+" range: %d to %d" %(t_init,t_init+t_range))
        plt.xticks(np.arange(0, len(corr)+1, 10))'''
        #######################################################
        



#final value finding

for i in range (12):
    
    #print("pos: %d Neg: %d pp:%d pn:%d pco:%f pcn:%f" %(memory_positive[i],memory_negative[i],picks[0,i],picks[1,i], memory_pos_corr[i], memory_neg_corr[i]))
    if (memory_pos_corr[i] >= memory_neg_corr[i]):
        msg[i]=memory_positive[i]
    else:
        msg[i]=memory_negative[i]

#print(picks[0,:])   
print("Positive Memory:", memory_positive)
#print(picks[1,:])   
print("Negative Memory:", memory_negative) 
print("Final Memory:", msg)
print("Please Run next step if final memory ins not verified")

Positive Memory: [103  30  58 206 180  30 157 119 231   5  33 119]
Negative Memory: [179 211 156 205 179 127 156 191 230   4  32 118]
Final Memory: [103 211  58 205 179  30 156 191 230   5  33 119]
Please Run next step if final memory ins not verified


## Note That, we found the memory as 
msg=[103, 211, 58, 205, 180, 30, 156, 191, 231, 5, 33, 119]

However, due to noise we might end up in mitchmatch in the founded value.
But we are cnfident enough that all the 12 elements are among these two arrays.

We need to verify the founded msg first. If it verify then its great.
Otherwise, we need to perform one more step. Which is try to verify all combination among two arrays for related index.

This will end up 2^12 = 4096. (2 arrays and 12 element) combination which is resonable.


In [13]:
#msg=[103, 211, 58, 205, 180, 30, 156, 191, 231, 5, 33, 119] #(Found Memory)
#Verifying
msg=list(msg)

print("Found Memory from high correlation:",msg)
target.simpleserial_write('a',bytearray(msg))
out_arr=target.simpleserial_read('r', 1)
print(out_arr)

Found Memory from high correlation: [103, 211, 58, 205, 179, 30, 156, 191, 230, 5, 33, 119]
CWbytearray(b'00')


In [14]:
mem_pos=memory_positive
mem_neg=memory_negative
#[103, 211, 58, 205, 180, 30, 156, 191, 231, 5, 33, 119] #Should be
mem_possible=[]
for i in trange (2**12):
    bin_str=format(i, '012b')
    #print(bin_str)
    msg=[0]*12
    for j in range (12):
        if (bin_str[j]=='0'):
            msg[j]=mem_pos[j]
        else:
            msg[j]=mem_neg[j]
    #print(msg)
    mem_possible.append(msg)
    
    #print(msg)
    target.simpleserial_write('a',bytearray(msg))
    out_arr=target.simpleserial_read('r', 1)
    
    if (list(out_arr) != [0]):
        print(list(out_arr))
        print(list(msg))
        break
        
#print(mem_possible)

  0%|          | 0/4096 [00:00<?, ?it/s]

[1]
[103, 211, 58, 205, 180, 30, 156, 191, 231, 5, 33, 119]


# Now we found the real memory.
Lets verify again.

In [15]:
#[103, 211, 58, 205, 180, 30, 156, 191, 231, 5, 33, 119] #Should be 
print("Found actual memory:",msg)
target.simpleserial_write('a',bytearray(msg))
out_arr=target.simpleserial_read('r', 1)
print("Goal:",out_arr)

Found actual memory: [103, 211, 58, 205, 180, 30, 156, 191, 231, 5, 33, 119]
Goal: CWbytearray(b'01')
